### Section 1: Preparation

In [1]:
import pandas as pd
import numpy as np
from datetime import date

import FundamentalAnalysis as fa

# import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
ticker = 'MU'
api_key = "99468a6d827666a6ee01f6788171dc40"

start_date = '1984-06-01'
end_date = '2022-12-31'

In [3]:
# Write single dataframe to excel sheet
def write_a_dataframe_to_excel(df, filename, sheetname):
    with pd.ExcelWriter(f'{filename}.xlsx', 
                        engine='xlsxwriter', 
                        datetime_format='yyyy') as writer:
        workbook = writer.book
        worksheet = workbook.add_worksheet(sheetname)
        writer.sheets[sheetname] = worksheet

        COLUMN = 0
        row = 0

        worksheet.write_string(row, COLUMN, df.name)
        row += 1
        df.to_excel(writer, sheet_name=sheetname,
                    startrow=row, startcol=COLUMN)
        
# Automatically write multiple dataframe to excel sheet
def write_dataframes_to_excel(dataframes, filename, sheetname):
    with pd.ExcelWriter(f'{filename}.xlsx', 
                        engine='xlsxwriter',
                        datetime_format='yyyy') as writer:
        workbook = writer.book
        worksheet = workbook.add_worksheet(sheetname)
        writer.sheets[sheetname] = worksheet

        COLUMN = 0
        row = 0

        for df in dataframes:
            worksheet.write_string(row, COLUMN, df.name)
            row += 1
            df.to_excel(writer, sheet_name=sheetname,
                        startrow=row, startcol=COLUMN,
                       float_format = '%.2f')
            row += df.shape[0] + 2

# Apply percentage format to number
def percentage_format(x):
    if pd.isna(x):
        return x
    else:
        return '{:.2%}'.format(x)

In [4]:
import yfinance as yf

# stock = yf.Ticker('AAPL')
stock = yf.download(ticker, start_date, end_date)      # get stock history data up to last year
stock_today = yf.download(ticker, end_date)            # get stock data for current year

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Section 2: Get price info

In [5]:
# Obtain last day in Dec. of each year
stock['year'] = stock.index.year
stock['month'] = stock.index.month
stock['day'] = stock.index.day
max_day_of_year = stock.loc[stock.index.month == 12].groupby('year').max()['day']

# Get historical stock price per year (year end price)
stock_close = stock.loc[(stock.index.month == 12) & (stock.index.day == max_day_of_year[stock.index.year])]
# stock_close = pd.concat([stock_close,stock_today])
stock_close = pd.concat([stock_close, stock_today.loc[stock_today.iloc[-1].name.strftime("%Y-%m-%d"):]])
stock_price = pd.DataFrame()
stock_price['Price'] = stock_close['Close']

# Formatting
stock_price = stock_price.sort_index(ascending=False).transpose()
# stock_price = pd.concat([stock_price.iloc[:-3].sort_index(ascending=False), stock_price.iloc[-3:]]).transpose()

# Get price percentage changes per year
returns = pd.DataFrame()
returns['PriceGrowth'] = stock_close['Close'].pct_change()

# Formatting
returns['PriceGrowth'] = returns['PriceGrowth'].apply(lambda x: percentage_format(x))
# returns = pd.concat([returns.iloc[:-3].sort_index(ascending=False), returns.iloc[-3:]]).transpose()
returns = returns.sort_index(ascending=False).transpose()

# Combine 2 rows
price_info = pd.concat([stock_price, returns])

# Change column names from timestamp format to 4 digit year
price_info.rename(columns=lambda s: str(s)[:4], inplace=True)
price_info

Date,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1993,1992,1991,1990,1989,1988,1987,1986,1985,1984
Price,59.99,49.98,93.15,75.18,53.78,31.73,41.12,21.92,14.16,35.01,...,4.65,1.8875,1.4125,1.0125,0.975,1.575,1.475,0.45,0.85,2.725
PriceGrowth,20.03%,-46.34%,23.90%,39.79%,69.49%,-22.84%,87.59%,54.80%,-59.55%,60.97%,...,146.36%,33.63%,39.51%,3.85%,-38.10%,6.78%,227.78%,-47.06%,-68.81%,NaN


### Section 3: Bring all finanical data needed for instrinsic value calculation

In [28]:
# Loan other finanical data
income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
cash_flow_statement_annually = fa.cash_flow_statement(ticker, api_key, period="annual")
growth_annually = fa.financial_statement_growth(ticker, api_key, period="annual")
key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
# dcf_annually = fa.discounted_cash_flow(ticker, api_key, period="annual")
entreprise_value = fa.enterprise(ticker, api_key)
balance_sheet_annually = fa.balance_sheet_statement(ticker, api_key, period="annual")

# Store financial data in results
results = pd.DataFrame()
results = pd.concat([income_statement_annually.loc['revenue'],
                     growth_annually.loc['revenueGrowth'], 
                     income_statement_annually.loc['netIncome'],
                     income_statement_annually.loc['eps'],
                     growth_annually.loc['epsgrowth'],
                     key_metrics_annually.loc['bookValuePerShare'],
                     growth_annually.loc['bookValueperShareGrowth'],
                     cash_flow_statement_annually.loc['freeCashFlow'],
                     key_metrics_annually.loc['freeCashFlowPerShare'],
                     growth_annually.loc['freeCashFlowGrowth'],
                     key_metrics_annually.loc['marketCap'],
                     key_metrics_annually.loc['enterpriseValue'],
                     key_metrics_annually.loc['peRatio'],
                     key_metrics_annually.loc['pbRatio'],
                     key_metrics_annually.loc['roe'],
                     balance_sheet_annually.loc['cashAndShortTermInvestments'],
                     balance_sheet_annually.loc['totalDebt'],
                     entreprise_value.loc['numberOfShares']
#                      dcf_annually.loc['DCF']
                    ], axis=1)

# Function to calculate eeRatio = enterpriseValue/netIncome
def calc_eeRatio(df):
    df['eeRatio'] = df['enterpriseValue'] / df['netIncome']
    return df

# Update eeRatio
results.rename({'pbRatio': 'eeRatio'}, axis='columns', inplace=True)
results = calc_eeRatio(results).transpose()

# results = results.transpose()
results = pd.concat([price_info, results])

results.loc['revenueGrowth'] = results.loc['revenueGrowth'].apply(lambda x: percentage_format(x))
results.loc['epsgrowth'] = results.loc['epsgrowth'].apply(lambda x: percentage_format(x))
results.loc['bookValueperShareGrowth'] = results.loc['bookValueperShareGrowth'].apply(lambda x: percentage_format(x))
results.loc['freeCashFlowGrowth'] = results.loc['freeCashFlowGrowth'].apply(lambda x: percentage_format(x))
results.loc['roe'] = results.loc['roe'].apply(lambda x: percentage_format(x))

### Section 4: Compound Annual Growth Rate (CAGR) for Price, Revenue, EPS & Free Cash Flow

In [29]:
# Calc Compound Annual Growth Rate (CAGR)
# def CAGR(first, last, periods):
#     return (last/first)**(1/periods)-1
def CAGR(first, last, periods):
    result = 0
    
    if first > 0 and last > 0:
        result = (last/first)**(1/periods)-1
    elif first < 0 and last < 0:
        result = -1 * ((abs(last)/abs(first))**(1/periods)-1)
    elif first < 0 and last > 0:
        result = ((last+2*abs(first))/abs(first))**(1/periods)-1
    elif first > 0 and last < 0:
        result = -1 * (((abs(last)+2*first)/first)**(1/periods)-1)
        
    return result


# Get last not null column
def earliest_col_notna(df,idx):
    for col in reversed(df.columns):
        if pd.notna(df.loc[idx][col]):
            return df.columns.get_loc(col)
        
# Calc CAGR (5/10/All years) for growth related columns
def calc_CAGR_for_col(df, idx):
    earliest_year = earliest_col_notna(df, idx)
    
    five_yrs_CAGR = np.nan
    ten_yrs_CAGR = np.nan
    All_CAGR = np.nan
    
    if earliest_year < 6:
        All_CAGR = CAGR(df.loc[idx][earliest_year], df.loc[idx][1], earliest_year-1)
    elif earliest_year < 11:
        five_yrs_CAGR = CAGR(df.loc[idx][6], df.loc[idx][1], 5)
        All_CAGR = CAGR(df.loc[idx][earliest_year], df.loc[idx][1], earliest_year-1)
    else:
        five_yrs_CAGR = CAGR(df.loc[idx][6], df.loc[idx][1], 5)
        ten_yrs_CAGR = CAGR(df.loc[idx][11], df.loc[idx][1], 10)
        All_CAGR = CAGR(df.loc[idx][earliest_year], df.loc[idx][1], earliest_year-1)
        
    return [five_yrs_CAGR, ten_yrs_CAGR, All_CAGR]

In [30]:
# Calc average returns from growth related columns
returns_lists = []

returns_lists.append(calc_CAGR_for_col(results,'Price'))
returns_lists.append(calc_CAGR_for_col(results,'revenue'))
returns_lists.append(calc_CAGR_for_col(results,'eps'))
returns_lists.append(calc_CAGR_for_col(results,'freeCashFlow'))

avg_returns = pd.DataFrame(returns_lists, 
             index=['PriceGrowth','revenueGrowth','epsgrowth','freeCashFlowGrowth'], 
             columns=['5 years Avg','10 years Avg','All years Avg'])

# Formatting
avg_returns['5 years Avg'] = avg_returns['5 years Avg'].apply(lambda x: percentage_format(x))
avg_returns['10 years Avg'] = avg_returns['10 years Avg'].apply(lambda x: percentage_format(x))
avg_returns['All years Avg'] = avg_returns['All years Avg'].apply(lambda x: percentage_format(x))

### Last section: Put all data together and save to excel

In [31]:
# price_info.name = ticker + ' price history'
results.name = ticker + ' Financial data'
avg_returns.name = ticker + ' Compound Annual Growth Rate (CAGR)'

dataframes = (results, avg_returns)
write_dataframes_to_excel(dataframes, 
                          'data\Stock fundamental analysis - '+ ticker + ' ' + date.today().strftime("%Y-%m"), 
                          ticker)